##### ResNet
##### 拼接ResNet块
<img src="https://s1.ax1x.com/2022/11/05/xObdXt.jpg" height=60% width=60%  />
<img src="https://img-blog.csdnimg.cn/img_convert/09a039c92e6a87a9d02e5973a8b368df.png" height=70% width=70%  />
<img src="https://img-blog.csdnimg.cn/ad2db8ace0b4402ab351e48fa289777e.png" height=15% width=15%  />

In [15]:
import torch
import torchvision
import torch.nn as nn
import d2l.torch
from torch.nn import functional as F
class Residual(nn.Module):
    def __init__(self,input_channels,output_channels,use_1x1conv=False,strides=1):
        super(Residual, self).__init__()
        self.conv1 = nn.Conv2d(input_channels,output_channels,kernel_size=3,padding=1,stride=strides)
        self.conv2 = nn.Conv2d(in_channels=output_channels,out_channels=output_channels,kernel_size=3,padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(in_channels=input_channels,out_channels=output_channels,kernel_size=1,stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(output_channels)
        self.bn2 = nn.BatchNorm2d(output_channels)
        self.relu = nn.ReLU(inplace=True)   
    def forward(self,x):
        Y = F.relu(self.bn1(self.conv1(x)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            x = self.conv3(x)
        Y += x
        return F.relu(Y)
#ResNet第一个模块跟GoogleNet第一个模块相同
b1 = nn.Sequential(nn.Conv2d(in_channels=1,out_channels=64,kernel_size=7,padding=3,stride=2),
                    nn.BatchNorm2d(64),
                    nn.ReLU(),
                    nn.MaxPool2d(kernel_size=3,padding=1,stride=2))
#定义一个ResNet块，通常包含两个残差块Residul块(也即是包含两个残差网络层)，一个ResNet块通常通道数加倍，尺寸形状高和宽减半，对应到由第一个残差块输出通道是输入通道两倍，尺寸大小减半，第二个残差块输入输出通道数相同，输入输出尺寸形状大小不变，但除开第二个ResNet块，因为第一个ResNet块将输入尺寸形状大小降低了4倍
#第一个不减半，因为前面已经做了两次减半
def resnet_block(input_channels,output_channels,num_residuls,first_block=False):
    block = []
    for i in range(num_residuls):
        if i==0 and not first_block:
            block.append(Residual(input_channels=input_channels,output_channels=output_channels,use_1x1conv=True,strides=2))
        else:
            block.append(Residual(input_channels=output_channels,output_channels=output_channels))
    return block
b2 = nn.Sequential(*resnet_block(64,64,2,first_block=True))
b3 = nn.Sequential(*resnet_block(64,128,2))
b4 = nn.Sequential(*resnet_block(128,256,2))
b5 = nn.Sequential(*resnet_block(256,512,2))

net = nn.Sequential(b1,b2,b3,b4,b5,nn.AdaptiveAvgPool2d((1,1)),
                    nn.Flatten(),
                    nn.Linear(in_features=512,out_features=10))

In [16]:
#查看每一层输出的通道数和形状尺寸大小
X = torch.randn(size=(1,1,224,224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__," output shape :\t",X.shape)

Sequential  output shape :	 torch.Size([1, 64, 56, 56])
Sequential  output shape :	 torch.Size([1, 64, 56, 56])
Sequential  output shape :	 torch.Size([1, 128, 28, 28])
Sequential  output shape :	 torch.Size([1, 256, 14, 14])
Sequential  output shape :	 torch.Size([1, 512, 7, 7])
AdaptiveAvgPool2d  output shape :	 torch.Size([1, 512, 1, 1])
Flatten  output shape :	 torch.Size([1, 512])
Linear  output shape :	 torch.Size([1, 10])


In [17]:
#训练测试
#ResNet模型训练和测试（学习率lr = 0.05,训练轮数为10轮，batch_size为64）
lr,num_epochs,batch_size = 0.05,10,64
train_iter,test_iter = d2l.torch.load_data_fashion_mnist(batch_size,resize=224)
d2l.torch.train_ch6(net,train_iter,test_iter,num_epochs,lr,device=d2l.torch.try_gpu())

  0%|          | 0/26421880 [00:00<?, ?it/s]